In [23]:
import pandas as pd

import requests
from tqdm import tqdm
import re
import difflib

def read_data(pkl_file_num):
    df = pd.read_pickle('G:/uiuc/timan/AIRC/data/parsed/'+pkl_file_num+'.pkl')
    return df

df1 = read_data('2022-15372')
df2 = read_data('2022-23918')

# print(doc1.head(),doc2.head())
print('Columns in each dataframe: ', df1.columns)

# Assuming your dataframes are named df1 and df2
filtered_df1 = df1[df1['tag'] == 'HD']
filtered_df2 = df2[df2['tag'] == 'HD']

# Merge the dataframes on the 'header' column
merged_df = pd.merge(filtered_df1, filtered_df2, on='header')

# Count the number of matching headers
matching_header_count = len(merged_df)

print(f"Number of matching headers for tag 'HD': {matching_header_count}")
print(merged_df)

# Find the common headers
common_headers = set(filtered_df1['header']).intersection(set(filtered_df2['header']))

# Initialize the dictionaries to store the concatenated text
header_text_dict_df1 = {}
header_text_dict_df2 = {}

# Iterate through the common headers
for header in common_headers:
    # Process for df1
    curr_index_df1 = df1[df1['header'] == header].index[0]
    next_index_df1 = df1.iloc[curr_index_df1 + 1:].loc[df1['header'].isin(common_headers)].index.min()

    if pd.isna(next_index_df1):
        text_df1 = ' '.join(df1.loc[(df1.index > curr_index_df1) & (df1['tag'] == 'P'), 'text'])
    else:
        text_df1 = ' '.join(df1.loc[(df1.index > curr_index_df1) & (df1.index < next_index_df1) & (df1['tag'] == 'P'), 'text'])

    header_text_dict_df1[header] = text_df1

    # Process for df2
    curr_index_df2 = df2[df2['header'] == header].index[0]
    next_index_df2 = df2.iloc[curr_index_df2 + 1:].loc[df2['header'].isin(common_headers)].index.min()

    if pd.isna(next_index_df2):
        text_df2 = ' '.join(df2.loc[(df2.index > curr_index_df2) & (df2['tag'] == 'P'), 'text'])
    else:
        text_df2 = ' '.join(df2.loc[(df2.index > curr_index_df2) & (df2.index < next_index_df2) & (df2['tag'] == 'P'), 'text'])

    header_text_dict_df2[header] = text_df2

print("Dictionary for df1:")
print(header_text_dict_df1)
print("\nDictionary for df2:")
print(header_text_dict_df2)


Columns in each dataframe:  Index(['xml_path', 'header', 'legal', 'tag', 'footnotes', 'numbering', 'text',
       'footnote', 'cfr_title', 'cfr_part', 'cfr_section', 'section_subject',
       'section'],
      dtype='object')
Number of matching headers for tag 'HD': 282
                      xml_path_x  \
0         /PRORULE/PREAMB/AGY/HD   
1         /PRORULE/PREAMB/ACT/HD   
2         /PRORULE/PREAMB/SUM/HD   
3     /PRORULE/PREAMB/EFFDATE/HD   
4         /PRORULE/PREAMB/ADD/HD   
..                           ...   
277  /PRORULE/SUPLINF/PART[6]/HD   
278  /PRORULE/SUPLINF/PART[7]/HD   
279  /PRORULE/SUPLINF/AUTH[7]/HD   
280  /PRORULE/SUPLINF/AUTH[8]/HD   
281  /PRORULE/SUPLINF/PART[8]/HD   

                                                header  legal_x tag_x  \
0                                           (AGENCY:,)    False    HD   
1                                           (ACTION:,)    False    HD   
2                                          (SUMMARY:,)    False    HD   
3   

In [47]:
# df1['header'] = df1['header'].apply(lambda x: tuple(y.lower() for y in x))
# df2['header'] = df2['header'].apply(lambda x: tuple(y.lower() for y in x))

df1['header'] = df1['header'].apply(lambda x: ' '.join(x).lower())
df2['header'] = df2['header'].apply(lambda x: ' '.join(x).lower())

In [58]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   xml_path         3491 non-null   object 
 1   header           3491 non-null   object 
 2   legal            3491 non-null   bool   
 3   tag              3491 non-null   object 
 4   footnotes        3130 non-null   object 
 5   numbering        631 non-null    object 
 6   text             3491 non-null   object 
 7   footnote         361 non-null    object 
 8   cfr_title        323 non-null    object 
 9   cfr_part         323 non-null    object 
 10  cfr_section      238 non-null    object 
 11  section_subject  238 non-null    object 
 12  section          0 non-null      float64
dtypes: bool(1), float64(1), object(11)
memory usage: 330.8+ KB


In [78]:
check_for_str='Table of Contents'

In [79]:
df1['header'].str.contains(check_for_str.lower())

0       False
1       False
2       False
3       False
4       False
        ...  
3486    False
3487    False
3488    False
3489    False
3490    False
Name: header, Length: 3491, dtype: bool

In [57]:
# count = (df1['header'].str.contains('Table of Contents'.lower())).sum()
# count

2

In [83]:
doc1_start_idx = df1.index[df1['header'].str.contains(check_for_str.lower())][0]
doc1_end_idx = df1.index[df1['header'].str.contains(check_for_str.lower())][-1]

df1.loc[doc1_start_idx:doc1_end_idx,['header','text']]


,header,text
63,supplementary information: table of contents,Table of Contents
64,supplementary information: table of contents,Summary and BackgroundA. Executive Summary of ...


In [84]:
doc2_start_idx = df2.index[df2['header'].str.contains(check_for_str.lower())][0]
doc2_end_idx = df2.index[df2['header'].str.contains(check_for_str.lower())][-1]

df2.loc[doc2_start_idx:doc2_end_idx,['header','text']]


,header,text
68,supplementary information: table of contents,Table of Contents
69,supplementary information: table of contents,Summary and BackgroundA. Executive Summary of ...


In [108]:
df1

,xml_path,header,legal,tag,footnotes,numbering,text,footnote,cfr_title,cfr_part,cfr_section,section_subject,section
0,/PRORULE/PREAMB/AGY/HD,agency:,False,HD,[],NaN,AGENCY:,NaN,NaN,NaN,NaN,NaN,NaN
1,/PRORULE/PREAMB/AGY/P,agency:,False,P,[],NaN,Centers for Medicare & Medicaid Services (CMS)...,NaN,NaN,NaN,NaN,NaN,NaN
2,/PRORULE/PREAMB/ACT/HD,action:,False,HD,[],NaN,ACTION:,NaN,NaN,NaN,NaN,NaN,NaN
3,/PRORULE/PREAMB/ACT/P,action:,False,P,[],NaN,Proposed rule.,NaN,NaN,NaN,NaN,NaN,NaN
4,/PRORULE/PREAMB/SUM/HD,summary:,False,HD,[],NaN,SUMMARY:,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,/PRORULE/SUPLINF/SECTION[32]/P[3],authority:,True,P,[],NaN,"(viii) Hospitals, including critical access ho...",NaN,42,424,424.518,Screening levels for Medicare providers and s...,NaN
3487,/PRORULE/SUPLINF/AMDPAR[45],authority:,True,AMDPAR,[],43.,Add § 424.575 to read as follows:,NaN,42,424,NaN,NaN,NaN
3488,/PRORULE/SUPLINF/SECTION[33],authority:,True,SECTION,[],NaN,§ 424.575 Rural emergency hospitals.(a) A rur...,NaN,42,424,NaN,NaN,NaN
3489,/PRORULE/SUPLINF/SECTION[33]/P[1],authority:,True,P,[],(a),A rural emergency hospital (as defined in § 48...,NaN,42,424,424.575,Rural emergency hospitals.,NaN


In [110]:
# create a new empty DataFrame
new_df1 = pd.DataFrame(columns=df1.columns)

content = ''
for index, row in df1.iterrows():
    if row['tag'] == 'HD':
        if content:
            new_row = {
                'xml_path': row['xml_path'], 'header':row['header'], 'legal':row['legal'], 'tag':'HD', 'footnotes':row['footnotes'], 'numbering':row['numbering'], 'text':row['text'],
       'footnote':row['footnote'], 'cfr_title':row['cfr_title'], 'cfr_part':row['cfr_part'], 'cfr_section':row['cfr_section'], 'section_subject':row['section_subject'],
       'section':row['section'], 'content' : content
            }
            new_df1 = new_df1.append(new_row, ignore_index=True)
        content = ''
    elif row['tag'] == 'P':
        content += row['text'] + '\n'
        
# add the last 'HD' tag and its content to the new DataFrame
new_row = {
                'xml_path': row['xml_path'], 'header':row['header'], 'legal':row['legal'], 'tag':'HD', 'footnotes':row['footnotes'], 'numbering':row['numbering'], 'text':row['text'],
       'footnote':row['footnote'], 'cfr_title':row['cfr_title'], 'cfr_part':row['cfr_part'], 'cfr_section':row['cfr_section'], 'section_subject':row['section_subject'],
       'section':row['section'], 'content' : content
            }
new_df1 = new_df1.append(new_row, ignore_index=True)

print(new_df1['content'])


0      Centers for Medicare & Medicaid Services (CMS)...
1                                       Proposed rule.\n
2      This proposed rule would revise the Medicare h...
3      To be assured consideration, comments must be ...
4      In commenting, please refer to file code CMS-1...
                             ...                        
486    42 U.S.C. 1302, 1395d(d), 1395f(b), 1395g, 139...
487    42 U.S.C. 1302 and 1395hh.\n* * *\nPre-propose...
488     42 U.S.C. 1302, 1395l(t), and 1395hh.\nAdditi...
489    Basis.\nScope.\nAs used in this subpart—\nRura...
490     42 U.S.C. 1302 and 1395hh.\n* * *\n* * *\n(vi...
Name: content, Length: 491, dtype: object


In [116]:
print(new_df1['content'][6])


Inspection of Public Comments:http://www.regulations.govRegulations.gov



In [135]:
df1.iloc[60:80][['text','tag']]

,text,tag
60,https://www.cms.gov/Medicare/Medicare-Fee-for-...,P
61,Current Procedural Terminology (CPT) Copyright...,HD
62,"Throughout this proposed rule, we use CPT code...",P
63,Table of Contents,HD
64,Summary and BackgroundA. Executive Summary of ...,EXTRACT
65,Summary and Background,HD
66,Executive Summary of This Document,HD
67,Purpose,HD
68,,P
69,Summary of the Major Provisions,HD
